# Creating more complex container image
We might want to use files from outside the container
* either by copying into the container images,
* or by making them visible within a running container from their existing location on the host system.

# Using scripts and files from outside the container
`cd` to `sum` directory in the downloaded zip file.
```console
(base) hell@Dell-Precision-T1600:~/Desktop/Docker/docker-intro/sum$ pwd
/home/hell/Desktop/Docker/docker-intro/sum
```

This directory has both `Dockerfile` and a python script called `sum.py`. We want to run the script using the container.

If we try this 
`docker container run alice/alpine-python python3 sum.py`
We will get an error message like this `python3: can't open file 'sum.py': [Errno 2] No such file or directory`.

The problem here is the container has a seperate filesystem/OS. That is why containers can't infect the host filesystem/OS. In order to use Python(inside the container) and our script (outside the container) we need to create a link between the directory on our computer and the container.

This link is called a **mount** and is just like mounting external drives to the computer. We need to have additional options to `docker container run` with `--mount type=bind,scource=${PWD},target=/temp`

Here, `${PWD}` is the current working directory, you can substitute the source directory where the file or python script is on the host computer. The target is the mount point inside the container.

## Types of mounts
`type` can be `bind`, `volume`, `tmfs`.
## Final command to run external files
`docker container run --mount type=bind,source=${PWD},target=/temp USERNAME/CONTAINER_IMAGE_NAME COMMAND_TO_RUN` 

For example,

`docker container run --mount type=bind,source=${PWD},target=/temp alice/alphine-python python3 /temp/sum.py`

Note that if we create any files in the `/temp` directory while the container is running, these files will appear on our host filesystem in the original directory and will stay there even when the container stops.

* `docker container run`: use Docker to run a container
* `--mount type=bind,source=${PWD},target=/temp`: connect my current working directory (`${PWD}`) as a folder inside the container called `/temp`
* `alice/alpine-python`: name of the container image to use to run the container
* `python3 /temp/sum.py`: what commands to run in the container

The same container can be ran interactively with a mount point as follows

`docker container run --mount type=bind,source=${PWD},target=/temp -it alice/alpine-python sh`

# Including your scripts and data within a container image
 Let’s assume that we’ve finished with our `sum.py` script and want to add it to the container image itself.

Add a newline into the `Dockerfile` to copy the file `COPY sum.py`. Then simply build the image as usual: ` docker image build -t alice/alpine-sum .`
### The Importance of Command Order in a Dockerfile
Docker builds things in layers. For example, if we put the `COPY` command before the `RUN` command, rebuilding needs to build the `RUN` layers again.

# If the files are on a GITHUB repo
Add this command in the end of the `Dockerfile`

`RUN git clone https://github.com/alice/mycode`

Similarly, the `wget` command can be used to download any file publicly available on the internet.

`RUN wget ftp://ftp.ncbi.nlm.nih.gov/blast/executables/blast+/2.10.0/ncbi-blast-2.10.0+-x64-linux.tar.gz`

# More fancy `Dockerfile` options 
## Make the `sum.py` script run automatically
To achieve the goal of having a command that always runs when a container is run from the container image and can be passed the arguments given on the command line, use the keyword `ENTRYPOINT` in the `Dockerfile`.

```console
FROM alpine

COPY sum.py /home
RUN apk add --update python3 py3-pip python3-dev

# Run the sum.py script as the default command and
# allow people to enter arguments for it
ENTRYPOINT ["python3", "/home/sum.py"]

# Give default arguments, in case none are supplied on
# the command-line
CMD ["10", "11"]
```

### Build and test it
```console
$ docker image build -t alpine-sum:v2 .
# Most of the time you are interested in the sum of 10 and 11:
$ docker container run alpine-sum:v2
# Sometimes you have more challenging calculations to do:
$ docker container run alpine-sum:v2 12 13 14
```
### Overriding the `ENTRYPOINT`
Sometimes you don’t want to run the image’s `ENTRYPOINT`

`docker container run -it alpine-sum:v2 /bin/sh`

will yield

`Please supply integer arguments`

So, use `--entrypoint` to manually override the entrypoint.

`docker container run -it --entrypoint /bin/sh alpine-sum:v2`

# Add the sum.py script to the PATH so you can run it directly
Modify `Dockerfile`
```console
FROM alpine

RUN apk add --update python3 py3-pip python3-dev

COPY sum.py /home
# set script permissions
RUN chmod +x /home/sum.py
# add /home folder to the PATH
ENV PATH /home:$PATH
```
### Build and test the image
```console
$ docker image build -t alpine-sum:v3 .
$ docker container run alpine-sum:v3 sum.py 1 2 3 4
```